In [16]:
import pandas as pd
import json

trips_df = pd.read_csv("./kanto_GTFS/trips.txt")
stop_times_df = pd.read_csv("./kanto_GTFS/stop_times.txt")

df = pd.merge(stop_times_df, trips_df, how="left", on="trip_id")

In [10]:
df = df[["trip_id", "arrival_time", "departure_time", "stop_id",
         "stop_sequence", "service_id", "trip_headsign", "trip_short_name", "shape_id"]]
df.to_csv("./kanto_GTFS/stop_times_.csv", index=False)
df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,trip_headsign,trip_short_name,shape_id
0,T0904000001,17:50:00,17:50:00,S090400014800200,0,S000001,ＪＲ岡本駅,平出シャトル,SP000001
1,T0904000001,17:50:00,17:50:00,S090400014900000,1,S000001,ＪＲ岡本駅,平出シャトル,SP000001
2,T0904000001,17:52:00,17:52:00,S090400015000000,2,S000001,ＪＲ岡本駅,平出シャトル,SP000001
3,T0904000001,17:52:00,17:52:00,S090400015100000,3,S000001,ＪＲ岡本駅,平出シャトル,SP000001
4,T0904000001,17:54:00,17:54:00,S090400015200000,4,S000001,ＪＲ岡本駅,平出シャトル,SP000001


In [29]:
res = []
for (trip_id, service_id, shape_id, trip_headsign, trip_short_name), g in df.groupby(by=["trip_id", "service_id", "shape_id", "trip_headsign", "trip_short_name"]):
    temp = []
    tt = []
    d = ""
    for _, r in g.iterrows():
        # 前のバス停の発時刻と、次のバス停の着時刻が同様の場合には、スキップする
        # ここはあとでなんとかしなきゃね
        if d == r["arrival_time"]:
            continue

        if r["stop_sequence"] == 0:
            tt.append({
                "d": r["departure_time"],
                "s": r["stop_id"]
            })
        else:
            tt.append({
                "a": r["arrival_time"],
                "d": r["departure_time"],
                "s": r["stop_id"]
            })
        d = r["departure_time"]
    res.append({
        "trip_id": trip_id,
        "service_id": service_id,
        "shape_id": shape_id,
        "trip_headsign": trip_headsign,
        "trip_short_name": trip_short_name,
        "tt": tt
    })


In [30]:
with open("timetable.json", "w") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)